<a href="https://colab.research.google.com/github/TirendazAcademy/Deep-Learning-with-TensorFlow/blob/main/Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 15.4 MB/s eta 0:00:00


# Importing Required Libraries

In [2]:
import keras_tuner as kt

In [3]:
import tensorflow as tf
from tensorflow import keras

# Loading the Dataset

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [5]:
img_train.shape

(60000, 28, 28)

In [6]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

# Model Building

In [7]:
def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,
                             sampling="log")
    optimizer = hp.Choice("optimizer", values=["sgd", "adam"])
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                  metrics=["accuracy"])
    return model

# Random Search Tuner

In [8]:
random_search_tuner = kt.RandomSearch(
  build_model, 
	objective="val_accuracy", 
	max_trials=5, 
	overwrite=True,
  directory="my_fashion_mnist", 
	project_name="my_rnd_search", 
	seed=42)

In [9]:
random_search_tuner.search(img_train, label_train, epochs=10, validation_split=0.2)

Trial 5 Complete [00h 01m 15s]
val_accuracy: 0.7333333492279053

Best val_accuracy So Far: 0.8601666688919067
Total elapsed time: 00h 06m 53s


## Taking a look at the best model

In [10]:
top3_models = random_search_tuner.get_best_models(num_models=3)

In [11]:
best_model = top3_models[0]

In [12]:
best_model

In [13]:
top3_params = random_search_tuner.get_best_hyperparameters(num_trials=3)

In [14]:
top3_params[0].values  

{'n_hidden': 8,
 'n_neurons': 37,
 'learning_rate': 0.008547485565344062,
 'optimizer': 'sgd'}

In [15]:
best_trial = random_search_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

Trial 3 summary
Hyperparameters:
n_hidden: 8
n_neurons: 37
learning_rate: 0.008547485565344062
optimizer: sgd
Score: 0.8601666688919067


In [16]:
best_trial = random_search_tuner.oracle.get_best_trials()[0]

In [17]:
best_trial.summary()

Trial 3 summary
Hyperparameters:
n_hidden: 8
n_neurons: 37
learning_rate: 0.008547485565344062
optimizer: sgd
Score: 0.8601666688919067


In [18]:
best_trial = random_search_tuner.oracle.get_best_trials()[0]
best_trial.summary()

Trial 3 summary
Hyperparameters:
n_hidden: 8
n_neurons: 37
learning_rate: 0.008547485565344062
optimizer: sgd
Score: 0.8601666688919067


In [19]:
best_model.fit(img_train, label_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 11s 4ms/step - loss: 0.3658 - accuracy: 0.8682
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3542 - accuracy: 0.8702
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3463 - accuracy: 0.8742
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3376 - accuracy: 0.8771
Epoch 5/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3297 - accuracy: 0.8801


In [20]:
test_loss, test_accuracy = best_model.evaluate(img_test, label_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.4056 - accuracy: 0.8558


# Model Building with HyperModel 

In [21]:
def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")
    optimizer = hp.Choice("optimizer", values=["sgd", "adam"])
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [22]:
class MyClassificationHyperModel(kt.HyperModel):
    def build(self, hp):
        return build_model(hp)
    def fit(self, hp, model, X, y, **kwargs):
        if hp.Boolean("normalize"):
            norm_layer = tf.keras.layers.Normalization()
            X = norm_layer(X)
        return model.fit(X, y, **kwargs)

In [23]:
from pathlib import Path
hyperband_tuner = kt.Hyperband(
    MyClassificationHyperModel(), objective="val_accuracy", seed=42,
    max_epochs=10, factor=3, hyperband_iterations=2,
    overwrite=True, directory="my_fashion_mnist", project_name="hyperband")

root_logdir = Path(hyperband_tuner.project_dir) / "tensorboard"
tensorboard_cb = tf.keras.callbacks.TensorBoard(root_logdir)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=2)
hyperband_tuner.search(img_train, label_train, epochs=10,
                       validation_split = 0.2, 
                       callbacks=[early_stopping_cb, tensorboard_cb])

Trial 60 Complete [00h 01m 17s]
val_accuracy: 0.8360000252723694

Best val_accuracy So Far: 0.8899166584014893
Total elapsed time: 00h 36m 23s


# BayesianOptimization Tuner

In [24]:
bayesian_opt_tuner = kt.BayesianOptimization(
    MyClassificationHyperModel(), 
    objective="val_accuracy", 
    seed=42,
    max_trials=10, 
    alpha=1e-4, 
    beta=2.6,
    overwrite=True, 
    directory="my_fashion_mnist", 
    project_name="bayesian_opt")

In [25]:
bayesian_opt_tuner.search(
	img_train, label_train, epochs=10, 
	validation_split = 0.2,  callbacks=[early_stopping_cb])

Trial 10 Complete [00h 01m 15s]
val_accuracy: 0.7518333196640015

Best val_accuracy So Far: 0.8859166502952576
Total elapsed time: 00h 11m 14s


## Resource:
- [Hands-on Machine Learning with TensorFlow](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/)

Let's connect [YouTube](http://youtube.com/tirendazacademy) | [Medium](http://tirendazacademy.medium.com) | [Twitter](http://twitter.com/tirendazacademy) | [Instagram](https://www.instagram.com/tirendazacademy) | [GitHub](http://github.com/tirendazacademy) | [Linkedin](https://www.linkedin.com/in/tirendaz-academy) | [Kaggle](https://www.kaggle.com/tirendazacademy) 😎